# Deep learning for complete beginners: recognising handwritten digits


In [1]:
from keras.datasets import mnist # subroutines for fetching the MNIST dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Dense # the two types of neural network layer we will be using
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values

Using TensorFlow backend.


In [2]:
batch_size = 128 # in each iteration, we consider 128 training examples at once
num_epochs = 20 # we iterate twenty times over the entire training set
hidden_size = 512

In [3]:
num_train = 60000 # there are 60000 training examples in MNIST
num_test = 10000 # there are 10000 test examples in MNIST

height, width, depth = 28, 28, 1 # MNIST images are 28x28 and greyscale
num_classes = 10 # there are 10 classes (1 per digit)

(X_train, y_train), (X_test, y_test) = mnist.load_data() # fetch MNIST data

X_train = X_train.reshape(num_train, height * width) # Flatten data to 1D
X_test = X_test.reshape(num_test, height * width) # Flatten data to 1D
X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= 255 # Normalise data to [0, 1] range
X_test /= 255 # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [4]:
inp = Input(shape=(height * width,)) # Our input is a 1D vector of size 784
hidden_1 = Dense(hidden_size, activation='relu')(inp) # First hidden ReLU layer
hidden_2 = Dense(hidden_size, activation='relu')(hidden_1) # Second hidden ReLU layer
out = Dense(num_classes, activation='softmax')(hidden_2) # Output softmax layer

model = Model(inp, out) # To define a model, just specify its input and output layers

In [5]:
model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy


In [8]:
model.fit(X_train, Y_train, # Train the model using the training set...
          batch_size=batch_size, nb_epoch=num_epochs,
          verbose=1, validation_split=0.1) # ...holding out 10% of the data for validation

/usr/lib/python3.4/dist-packages/ipykernel_launcher.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  This is separate from the ipykernel package so we can avoid doing imports until


Train on 54000 samples, validate on 6000 samples
Epoch 1/20
54000/54000 [==============================] - 3s - loss: 0.2350 - acc: 0.9308 - val_loss: 0.0966 - val_acc: 0.9697
Epoch 2/20
54000/54000 [==============================] - 2s - loss: 0.0827 - acc: 0.9746 - val_loss: 0.0834 - val_acc: 0.9745
Epoch 3/20
54000/54000 [==============================] - 2s - loss: 0.0538 - acc: 0.9831 - val_loss: 0.0694 - val_acc: 0.9807
Epoch 4/20
54000/54000 [==============================] - 2s - loss: 0.0372 - acc: 0.9879 - val_loss: 0.0775 - val_acc: 0.9740
Epoch 5/20
54000/54000 [==============================] - 2s - loss: 0.0278 - acc: 0.9909 - val_loss: 0.0695 - val_acc: 0.9820
Epoch 6/20
54000/54000 [==============================] - 2s - loss: 0.0188 - acc: 0.9937 - val_loss: 0.0726 - val_acc: 0.9808
Epoch 7/20
54000/54000 [==============================] - 2s - loss: 0.0241 - acc: 0.9918 - val_loss: 0.0840 - val_acc: 0.9777
Epoch 8/20
54000/54000 [==============================] - 2s -

In [9]:
model.evaluate(X_test, Y_test, verbose=1) # Evaluate the trained model on the test set!


 9632/10000 [===========================>..] - ETA: 0s

[0.092943688172849537, 0.98080000000000001]

# Deep learning for complete beginners: convolutional neural networks with keras

In [6]:
from keras.callbacks import TensorBoard
cb = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=32, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [7]:
from keras.datasets import cifar10 # subroutines for fetching the CIFAR-10 dataset
from keras.models import Model # basic class for specifying and training a neural network
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils # utilities for one-hot encoding of ground truth values
import numpy as np

In [12]:
batch_size = 32 # in each iteration, we consider 32 training examples at once
num_epochs = 50
# we iterate 200 times over the entire training set
kernel_size = 3 # we will use 3x3 kernels throughout
pool_size = 2 # we will use 2x2 pooling throughout
conv_depth_1 = 32 # we will initially have 32 kernels per conv. layer...
conv_depth_2 = 64 # ...switching to 64 after the first pooling layer
drop_prob_1 = 0.25 # dropout after pooling with probability 0.25
drop_prob_2 = 0.5 # dropout in the FC layer with probability 0.5
hidden_size = 512 # the FC layer will have 512 neurons

In [13]:
X_train.shape

(50000, 32, 32, 3)

In [14]:
(X_train, y_train), (X_test, y_test) = cifar10.load_data() # fetch CIFAR-10 data

num_train, height, width, depth = X_train.shape # there are 50000 training examples in CIFAR-10 
num_test = X_test.shape[0] # there are 10000 test examples in CIFAR-10
num_classes = np.unique(y_train).shape[0] # there are 10 image classes

X_train = X_train.astype('float32') 
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Normalise data to [0, 1] range
X_test /= np.max(X_test) # Normalise data to [0, 1] range

Y_train = np_utils.to_categorical(y_train, num_classes) # One-hot encode the labels
Y_test = np_utils.to_categorical(y_test, num_classes) # One-hot encode the labels

In [15]:
inp = Input(shape=(height, width, depth)) # depth goes last in TensorFlow back-end (first in Theano)
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(inp)
conv_2 = Convolution2D(conv_depth_1, (kernel_size, kernel_size), padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, (kernel_size, kernel_size), padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_4)
drop_2 = Dropout(drop_prob_1)(pool_2)
# Now flatten to 1D, apply FC -> ReLU (with dropout) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)
drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)

model = Model(inputs=inp, outputs=out) # To define a model, just specify its input and output layers

model.compile(loss='categorical_crossentropy', # using the cross-entropy loss function
              optimizer='adam', # using the Adam optimiser
              metrics=['accuracy']) # reporting the accuracy

model.fit(X_train, Y_train,                # Train the model using the training set...
          batch_size=batch_size, epochs=num_epochs,
          verbose=1, validation_split=0.1, 
          callbacks=[cb]) # ...holding out 10% of the data for validation
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!

Train on 45000 samples, validate on 5000 samples
Epoch 1/50
45000/45000 [==============================] - 23s - loss: 1.5450 - acc: 0.4364 - val_loss: 1.1246 - val_acc: 0.5990
Epoch 2/50
45000/45000 [==============================] - 23s - loss: 1.1645 - acc: 0.5862 - val_loss: 0.9592 - val_acc: 0.6662
Epoch 3/50
45000/45000 [==============================] - 23s - loss: 1.0145 - acc: 0.6413 - val_loss: 0.8937 - val_acc: 0.6874
Epoch 4/50
45000/45000 [==============================] - 23s - loss: 0.9149 - acc: 0.6758 - val_loss: 0.9001 - val_acc: 0.6914
Epoch 5/50
45000/45000 [==============================] - 22s - loss: 0.8380 - acc: 0.7058 - val_loss: 0.7528 - val_acc: 0.7374
Epoch 6/50
45000/45000 [==============================] - 22s - loss: 0.7842 - acc: 0.7252 - val_loss: 0.7723 - val_acc: 0.7262
Epoch 7/50
45000/45000 [==============================] - 22s - loss: 0.7322 - acc: 0.7425 - val_loss: 0.7089 - val_acc: 0.7520
Epoch 8/50
45000/45000 [==============================]

[0.77678922514915472, 0.77749999999999997]

In [19]:
model.evaluate(X_test, Y_test, verbose=1)  # Evaluate the trained model on the test set!

 9888/10000 [============================>.] - ETA: 0s

[0.90125298643112184, 0.78500000000000003]